In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [2]:
# Input:
Den_h2so4 = 1.8 # g/cm³

tmp = [240] # temperature [K]
tmp = np.array(tmp)

aw = [0.0, 0.5, 0.7, 0.8, 0.9, 0.95, 0.99] # = relative humidity / 100

# [um]
Rdry = [0.00039422487,0.0004961,0.0006251,0.0007875,0.0009922,0.0012501,0.0015750,0.0019844,\
        0.0025002,0.0031501,0.0039689,0.0050004,0.0063002,0.0079377,0.0100009,0.0126003,\
        0.0158754,0.0200018,0.0252006,0.0317508,0.0400035,0.0504013,0.0635016,0.0800070,\
        0.1008026,0.1270033,0.1600141,0.2016051,0.2540065,0.3200282,0.4032103,0.5080131,\
        0.6400564,0.8064206,1.0160263,1.2801129,1.6128411,2.0320525,2.5602257,3.2256823]
Rdry = np.array(Rdry)

In [3]:
# !        calculation of particle density
# !        requires Temperature (temp) and acid mass fraction (so4mfrac)
# !
# !
# !---->Vehkamaeki et al., 2002  (JGR, doi:10.1029/2002JD002184)

def density(temp,so4mfrac):
    
    a1= 0.7681724
    a2= 2.184714
    a3= 7.163002
    a4=-44.31447
    a5= 88.74606
    a6=-75.73729
    a7= 23.43228
    
    b1= 1.808225e-3
    b2=-9.294656e-3
    b3=-3.742148e-2
    b4= 2.565321e-1
    b5=-5.362872e-1
    b6= 4.857736e-1
    b7=-1.629592e-1
     
    c1=-3.478524e-6
    c2= 1.335867e-5
    c3= 5.195706e-5
    c4=-3.717636e-4
    c5= 7.990811e-4
    c6=-7.458060e-4
    c7= 2.581390e-4
    
 
    so4m2=so4mfrac*so4mfrac
    so4m3=so4mfrac*so4m2
    so4m4=so4mfrac*so4m3
    so4m5=so4mfrac*so4m4
    so4m6=so4mfrac*so4m5

    a=a1+a2*so4mfrac+a3*so4m2+a4*so4m3 +a5*so4m4+a6*so4m5+a7*so4m6
    b=b1+b2*so4mfrac+b3*so4m2+b4*so4m3 +b5*so4m4+b6*so4m5+b7*so4m6
    c=c1+c2*so4mfrac+c3*so4m2+c4*so4m3 +c5*so4m4+c6*so4m5+c7*so4m6
      
    density=(a+b*temp+c*temp*temp) # units are g/cm**3
        
    return density

In [ ]:
data = np.zeros((len(aw)*len(Rdry), 4)) # RH_%, wp_%, Rdry_um, Rwet_um

In [25]:
# print(f'Rdry={Rdry[:]}')

Rwet = np.zeros((len(tmp), len(aw), len(Rdry)))
P_RH = np.zeros((len(tmp), len(aw), len(Rdry)))
P_wp = np.zeros((len(tmp), len(aw), len(Rdry)))

for i in range(len(tmp)):
    for k in range(len(aw)):
        
        if abs(aw[k])<1e-10:
            print("The wet radius is same as the dry radius")
            print("Just stop and no need to run the rest!")
            Rwet[i,k,:] = Rdry[:]
            P_RH[i,k,:] = aw[k]*100
            P_wp[i,k,:] = 99.0
            
        else:
            if aw[k]>1.0: aw[k]=1.0
            if aw[k]<=0.05:
                y1=12.372089320*aw[k]**(-0.16125516114) -30.490657554*aw[k] -2.1133114241
                y2=13.455394705*aw[k]**(-0.19213122550) -34.285174607*aw[k] -1.7620073078
            elif aw[k]<=0.85:
                y1=11.820654354*aw[k]**(-0.20786404244) -4.8073063730*aw[k] -5.1727540348
                y2=12.891938068*aw[k]**(-0.23233847708) -6.4261237757*aw[k] -4.9005471319
            else:
                y1=-180.06541028*aw[k]**(-0.38601102592) -93.317846778*aw[k] +273.88132245
                y2=-176.95814097*aw[k]**(-0.36257048154) -90.469744201*aw[k] +267.45509988

            sulfmolal=y1+((tmp[i]-190.)*(y2-y1)/70.)
        
            wp=9800.*sulfmolal/(98.*sulfmolal+1000.) # weight percentage
        
            if wp<15.0: wp=15.0
            if wp>99.0: wp=99.0
            if wp<15.0 or wp>100.0:
                print( 'weight percent= ',wp,  ', T=',tmp[i], ', RH(aw)=', aw[k])
                print('*** Stupid Value of Weight Percent ***')
             
                
            # mass fraction
            wpp=wp*0.01
            # mole fraction
            xa=18.*wpp/(18.*wpp+98.*(1.-wpp))
        
        
            den = density(tmp[i],wpp)
        
        
            Vdry = 4/3 *np.pi * Rdry[:]**3
            Mdry = Den_h2so4*Vdry[:]

            Mwet = Mdry[:]/wpp
            Vwet = Mwet[:]/den
        
            Rwet[i,k,:] = (Vwet[:] / (4/3 *np.pi))**(1/3)
            P_RH[i,k,:] = aw[k]*100
            P_wp[i,k,:] = wp
        
#             data = np.zeros((len(aw)*len(Rdry), 4)) # RH_%, wp_%, Rdry_um, Rwet_um
            
            
#             print( f'temp={tmp[i]:.1f}K, RH={aw[k]*100:.1f}%, wp={wp:.3f} %, xa={xa:.3f}, den={den:.3f} gm/cm**3' )
#             print(f'Rwet={Rwet[i,k,:]}')
#             print('')

The wet radius is same as the dry radius
Just stop and no need to run the rest!


In [38]:

# Rwet = np.zeros((len(tmp), len(aw), len(Rdry)))
data = np.zeros((len(Rdry)*len(aw), 4)) # RH_%, wp_%, Rdry_um, Rwet_um


for j in range(len(Rdry)):
    for i in range(len(aw)):
        index = j*len(aw) + i
        data[index,0] = Rdry[j]
        data[index,1] = P_RH[0,i,j]
        data[index,2] = P_wp[0,i,j]
        data[index,3] = Rwet[0,i,j]
        
df = pd.DataFrame(data)
df.columns =["Rdry_um", "RH_%", "wp_%", "Rwet_um"]

df.to_csv('Rdry_RH_wp_Rwet.csv')

In [37]:
df

Rdry_um  RH_%       wp_%   Rwet_um
0    0.000394   0.0  99.000000  0.000394
1    0.000394  50.0  39.842581  0.000591
2    0.000394  70.0  30.549615  0.000659
3    0.000394  80.0  25.469146  0.000709
4    0.000394  90.0  17.952168  0.000811
..        ...   ...        ...       ...
275  3.225682  70.0  30.549615  5.393621
276  3.225682  80.0  25.469146  5.798730
277  3.225682  90.0  17.952168  6.636533
278  3.225682  95.0  15.000000  7.099407
279  3.225682  99.0  15.000000  7.099407

[280 rows x 4 columns]